# Mini Problem Set: Diagnosis
## Implicats and Implicants for Constraint Programming (100 points)

1. [Introduction](#introduction)
2. [Part 1 - Tree method](#part-1)
3. [Part 2 - Prime implicate method](#part-2)
    1. [Problem 2 - Prune prime implicant children](#problem-2-1)
    2. [Problem 1 - Prune minimal conflict children](#problem-2-2)
    3. [Problem 3 - Skip satisfiable candidate](#problem-2-3)
    4. [Problem 4 - Full algorithm](#problem-2-4)

Make sure you load the dependencies below by highlighting the cell below and pressing Shift + Enter.

In [8]:
%load_ext autoreload
%autoreload 2
from propositional_state_logic import *
from sat_solver import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Introduction <a id="introduction"/>
</a>

In this problem set, you'll implement...

## Part 1 - Tree method <a id="part-1"/>
</a>

## Part 2 - Prime implicate method <a id="part-2"/>
</a>

In [9]:
p = Problem()

# Define the variables for the mini thruster problem with variables: T1, R1, and P3. Returns a Variable object.
# Thrust: T1
T1 = p.add_variable('thrustoutput', type='finite_domain', domain=['thrust', 'nothrust'])
# Thruster: R1
R1 = p.add_variable('thruster', type='finite_domain', domain=['on', 'off'])
# Pressure before the thruster: P3
P3 = p.add_variable('pressure', type='finite_domain', domain=['high', 'low'])

# Add the theory / problem constraints.
# The thruster only outputs thrust when it is on and when the input from P3 is high
p.add_constraint('thruster=on & pressure=high => thrustoutput=thrust')
p.add_constraint('thruster=on & pressure=low => thrustoutput=nothrust')
p.add_constraint('thruster=off => thrustoutput=nothrust')

# Prints out constraints nicely in LaTeX, so you can check them.
display_constraints(p)

Constraints:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [10]:
thruster_model = {
    frozenset([T1.get_assignment('thrust')]) : {
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('thrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('thrust'), P3.get_assignment('low')]) : {},
        },
    
    frozenset([T1.get_assignment('nothrust')]) : {
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off')]) : {
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('high')]) : {},
        frozenset([T1.get_assignment('nothrust'), P3.get_assignment('low')]) : {},
        },
    
    frozenset([R1.get_assignment('on')]) : {
            frozenset([R1.get_assignment('on'), P3.get_assignment('high')]) : {},
            frozenset([R1.get_assignment('on'), P3.get_assignment('low')]) : {},
            },
        frozenset([R1.get_assignment('off')]) : {
            frozenset([R1.get_assignment('off'), P3.get_assignment('high')]) : {},
            frozenset([R1.get_assignment('off'), P3.get_assignment('low')]) : {},
            },
    
    frozenset([P3.get_assignment('high')]) : {},
    
    frozenset([P3.get_assignment('low')]) : {},
    }

### Problem 1 - Prune prime implicant children <a id="problem-2-1"/>
</a>



In [13]:
sat = SATSolver(p)

In [11]:
def is_conflict(sat, candidate):
    return not sat.check_consistency(candidate)[0]


In [17]:
is_conflict(sat, frozenset([T1.get_assignment('thrust'), R1.get_assignment('off')]))

True

In [16]:
is_conflict(sat, frozenset([T1.get_assignment('thrust'), R1.get_assignment('on'), P3.get_assignment('high')]))

False

In [ ]:
def iterate_tree(tree):
    for 


In [20]:
def is_valid(sat, candidate_dict:
    if is_conflict(sat, candidate_dict):
        return False
    elif children == {}:
        return True
    else:
        return sum([is_valid(sat, child) for child in children]) == len([is_valid(child) for child in children])

In [21]:
is_valid(sat, frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on')]),
         [frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('high')]),
            frozenset([T1.get_assignment('nothrust'), R1.get_assignment('on'), P3.get_assignment('low')])])

TypeError: is_valid() missing 1 required positional argument: 'children'

### Problem 2 - Prune minimal conflict children <a id="problem-2-2"/>
</a>



### Problem 3 - Skip satisfiable candidate <a id="problem-2-3"/>
</a>

### Problem 4 - Full algorithm <a id="problem-2-4"/>
</a>